In [ ]:
from numpwd.integrate.numeric import ExpressionMap
from numpwd.densities.h5 import read_h5
from numpwd.operators.base import Operator
from numpwd.convolution.channels import get_channel_overlap_indices

from pandas import DataFrame
import pandas as pd

from sympy import sympify
import numpy as np
import os

## Read density

In [ ]:
DATA = os.path.join(os.environ["HOME"], "data", "nuc", "4he", "av18")
f = "compton-dens-4he-av18-empot-no3nf-om=0.00E+00-th=1.80E+02-nx=16-nphi=16-np12=np34=28+8-np3=36+8-nq4=nq=36+6-j12max=5-lmax=6-lsummax=14-tau4max=0-rho.h5"

In [ ]:
dens = read_h5(os.path.join(DATA, f))

In [ ]:
dens.channels

This is a pure contact interaction. So $l_i = l_o = 0$

## Operator A

In [ ]:
def get_contact_frame(dens):
    id_dens = dens.channels.query(
        "mj_o == mj_i and l_o == l_i == 0 and s_o == s_i and j_o == j_i and t_o == t_i and mt_o == mt_i"
    ).index
    weight = dens.p ** 2 * dens.wp
    weight = weight.reshape(1, len(weight), 1) * weight.reshape(1, 1, len(weight))
    channel_res = np.sum(dens.matrix[id_dens] * weight, axis=(1, 2))
    df = dens.channels.copy().loc[id_dens]
    df["res"] = channel_res
    return df

In [ ]:
df = get_contact_frame(dens)
df.head()

In [ ]:
def op_a(df, k2):
    return df["res"].sum() * k2


op_a(df, 1)

In [ ]:
def op_b(df, k2):
    out = 0
    grouped = df.groupby("s_o")
    for so, group in grouped:
        out += group["res"].sum() * k2 * (-3 if so == 0 else 1)
    return out


op_b(df, 1)

The c factor is effectively
$$ (-1)^{m_s + 1} $$

Thus the $(ls)jm_j$ contraction becomes
$$
    \sum_{m_l m_s}
    < l m_l, s m_s | j m_j>
    < l m_l, s m_s | j' m_j'> (-1)^{m_s +1}
$$
$$
    -\delta_{j j'}\delta_{m_j m_j'}\delta_{s 0}
    +\delta_{s 1}
    \left(\sum_{m_l m_s}
    < l m_l, 1 m_s | j m_j>
    < l m_l, 1 m_s | j' m_j'>
    -2
    < l m_j, 1 0 | j m_j>
    < l m_j, 1 0 | j' m_j>\right)
$$
and using that $l=0$
$$
    < 0 0, 1 0 | j 0>
    =
    \delta_{j1}
$$

In [ ]:
def op_c(df, k2):
    out = 0
    grouped = df.groupby(["s_o", "mj_o"])
    for (s, mj), group in grouped:
        if s == 0:
            fact = -1
        elif s == 1 and mj != 0:
            fact = 1
        elif s == 1 and mj == 0:
            fact = -1
        out += group["res"].sum() * k2 * fact
    return out


op_c(df, 1)